### Import des bibliothèques requises

In [1]:
import spotipy, threading, itertools, time, datetime, os, numpy as np
import pandas as pd

from collections import defaultdict
from spotipy.oauth2 import SpotifyClientCredentials


### Initialisation du Client Spotipy => 19 apps
   - ##### Accounts used: zoltar (18 apps) + unknown (1 app)

In [2]:
client_id = ['02dc5f25b96f4315a7182583264c75ba','7672d38906344d66bf7bc384e290950a','82a2058cd6d0435bb464648fdcf5dc8a','e567bb78a9ea46deb253263bfb4b0cc8','810337a602d14bd884429991ad417eba','3e931759ccea4d49bcaf44a1e4f6059b','6b353a3c6abe48fe86564875faecbb09','73ebc19135a54e4c9d9d7bae6a798aea','085733e14f894738bf40334443020126', 'bf7349672eab4050bf4567ed7c6732ec', 'dbf80683949a456592837fe493b579a7', 'b6f601f082f7491aa9c4ab7271fae6fd', '0464cab38a1a4fb89c8678f6b1e27e81', '65df35b84b8a47d2b66e21a2a8e39dd3', 'f61219e4380f4b8796e9f4d224da5494', '3c51ca3a9e314bd5aeb608d22e369efd', '3f3a6839a5de407ab16c87ba3ba8cd68', 'a8461dc61ff44abd83f94a4a07de530f', '7a06f1d458ac41f8b25577e02ec30d62']
client_secret = ['1708c607af8844e2aa4c8948d5098422','f47c320326924302a7e17995cd7b6aad','88549dde8bd04284a8c380d34fba4468','bebddff9b80841529c008b87fb9bf1e1','1eef810711464eafa0079ce3f48ae921','a1925f6f589f43e7b4456665ef0feb21','3f598249bd864a24bd07344adf9d9ac8','2ddf3a4559884ce6a2b438a0ec3c2937','62d8f8eccfdf49d9bf8710eb47d2a34c', 'bda8d2acbf974cd5932f00a6a763fad3', 'ac2a80f3cde94feb8c4b886cfd556cfd', 'fc0e21ebc5974886acc6ba14d4676c02', '58c42925f46941c6b7ca21cb157908b6', '90e3ed69a2f24735a5d4710bc6a377fa', 'cb4cb64077834ba2bae059bf983dd47f', '174b4c06e0974b01bbf7294d11c9bd7f', '03410f4b5165403b8219d14cc9550605', 'ed2e1d6898834e4bb649df080e5474bf', '2f10949260a449968cb0fa13f1e5a4c9']

spotify_client_credentials = []
spotify_client = []

for i in range(len(client_id)):
      spotify_client_credentials.append(SpotifyClientCredentials(client_id=client_id[i], client_secret=client_secret[i]))

for cred in spotify_client_credentials:                                                    
      spotify_client.append(spotipy.Spotify(client_credentials_manager = spotify_client_credentials[0]))

### Initialisation du Client Spotipy => 20 apps
   - ##### Accounts used: ncp (20 apps)

In [59]:
accounts = pd.read_csv('accounts.csv')

client_id = list(accounts['ID'])
client_secret = list(accounts['SECRET'])

spotify_client_credentials = []
spotify_client = []

for i in range(len(client_id)):
      spotify_client_credentials.append(SpotifyClientCredentials(client_id=client_id[i], client_secret=client_secret[i]))

for cred in spotify_client_credentials:                                                    
      spotify_client.append(spotipy.Spotify(client_credentials_manager = spotify_client_credentials[0]))

### Chargement des données préalablement extraites (source : MusicBrainz)

In [ ]:
input_data = pd.read_csv('../data/data_110k_lignes.csv', usecols=['artist_name', 'album_id'])
input_data

### Retrieve Data from Spotify API

In [61]:
# Initialization of the 2 main variables 
# 1) track_db_without_af >> populated with the result of the search endpoint
# 2) af_db >> populated with the result of the audio_features endpoint

track_db_without_af = []

# Input data - Select a slice for testing => ie. dataframe[:100]

input_data = df_year_genre

# What are the two search criterias used?

input_main_data_type = 'year'
input_secondary_data_type = 'genre'

# The hipster_mode forces the search endpoint to
# return albums with a popularity <= 10
#
# Please note: it only works for album (type='album')
#
# Possible values:  ' tag:hipster'  >> TRUE
#                   ''              >> FALSE

hipster_mode = ''

# Data slicing setup: each thread will handle one slice of the input_data

number_of_threads = len(spotify_client)
slice_size = len(input_data) // (number_of_threads)
slice_limit = []
thread = []

for i in range(number_of_threads + 1):
    if i == 0:
        slice_limit.append(0)
    elif i != number_of_threads:
        slice_limit.append(i*slice_size)
    else:
        slice_limit.append(len(input_data))

# Optional:
# - maximum number of consecutive requests?
# - time to sleep once that maximum is reached?

NOMAX = 1e4
MAX = 2
SLEEPTIME = 2

def get_audio_features(track_ids, thread_number):
    audio_features = []

    if len(track_ids) > 100:
        print("TOO MANY TRACK IDS")
        return audio_features
    else:
        audio_features = spotify_client[thread_number].audio_features(track_ids)
        audio_features = [item for item in audio_features if item is not None]

    for idx, item in enumerate(audio_features):
        item['track_id'] = item.pop('uri')
        item.update({'track_id': item['track_id'].split(':')[-1]})

    return audio_features

def get_audio_analysis(track_id, thread_number):

    audio_analysis = spotify_client[thread_number].audio_analysis(track_id)
    # audio_features = [item for item in audio_features if item is not None]

    # for idx, item in enumerate(audio_features):
    #     item['track_id'] = item.pop('uri')
    #     item.update({'track_id': item['track_id'].split(':')[-1]})

    return audio_analysis

def launch_thread(start, end, thread_number, one_year_limit=False):
    try:
        max_consecutive_requests = MAX

        # TBC (To Be Continued)
        for round in range(20):
            for index in range(start, end):
                row = input_data.iloc[index]
                
                if (index+1) % max_consecutive_requests == 1:
                    time.sleep(SLEEPTIME)
                
                print(input_secondary_data_type + ": " + row[input_secondary_data_type], input_main_data_type + ": " + row[input_main_data_type])
                print("Thread n°", thread_number + 1, " -- Round n°", round+1, " -- Request n°", index+1)

                print(input_secondary_data_type + ":" + row[input_secondary_data_type] + ' ' + input_main_data_type + ': ' + row[input_main_data_type])
                
                track_results = spotify_client[thread_number].search(q=input_secondary_data_type + ':' + \
                                                                    row[input_secondary_data_type] + ' ' + \
                                                                    input_main_data_type + ':' + \
                                                                    row[input_main_data_type] + \
                                                                    hipster_mode, type='track',
                                                                    limit=50,
                                                                    offset=round*50)
                                                                    # TBC (To Be Continued)
                                                                    # offset=(round+9)*50)

                if track_results['tracks']['total'] == 0:
                    continue

                track_ids_for_batch_AF_request = []

                for i, t in enumerate(track_results['tracks']['items']):

                    track_info = dict(track_id = t['id'], track_name = t['name'], release_date = t['album']['release_date'],
                                    popularity = t['popularity'], duration_ms = t['duration_ms'], explicit = t['explicit'],
                                    is_local = t['is_local'], track_number = t['track_number'], genre = row[input_secondary_data_type])

                    if one_year_limit:
                        if len(track_info['release_date']) == 10:
                            if datetime.datetime.strptime(track_info['release_date'], '%Y-%m-%d').date().year == 2022:
                                track_db_without_af.append(track_info)
                                track_ids_for_batch_AF_request.append(t['id'])
                        elif len(track_info['release_date']) == 4:
                            if int(track_info['release_date']) == 2022:
                                track_db_without_af.append(track_info)
                                track_ids_for_batch_AF_request.append(t['id'])
                        elif len(track_info['release_date']) == 7:
                            if datetime.datetime.strptime(track_info['release_date'], '%Y-%m').date().year == 2022:
                                track_db_without_af.append(track_info)
                                track_ids_for_batch_AF_request.append(t['id'])
                        else:
                            # TBC (To Be Continued)
                            print("CKOICETRUC ? =>", track_info['release_date'])
                    else:
                        track_db_without_af.append(track_info)
                        track_ids_for_batch_AF_request.append(t['id'])

                if len(track_ids_for_batch_AF_request) > 100:
                    raise Exception("Expected - Development Exception: more than 100 audio features to recover (feature in development)")
                else:
                    af_db.extend(get_audio_features(track_ids_for_batch_AF_request, thread_number))

            pd.DataFrame(track_db_without_af).to_csv('data-out/musicbrainz/temp_backup/track_db_without_af___thread' + str(thread_number) + '.csv')
            pd.DataFrame(af_db).to_csv('data-out/musicbrainz/temp_backup/af_db___thread' + str(thread_number) + '.csv')

    except Exception as e:
        print("Exception raised for thread N°", thread_number,":", e)

# Launch the multithreading
for i in range(number_of_threads):
    thread.append(threading.Thread(target=launch_thread, args=(slice_limit[i], slice_limit[i+1], i)))
    thread[i].start()

# Wait for completion
for i in range(number_of_threads):
    thread[i].join()

genre: mandopop year: 2022
Thread n° 13  -- Round n° 1  -- Request n° 73
genre:mandopop year: 2022
genre: reggae year: 2022
Thread n° 17  -- Round n° 1  -- Request n° 97
genre:reggae year: 2022
genre: progressive-house year: 2022
Thread n° 16  -- Round n° 1  -- Request n° 91
genre:progressive-house year: 2022
genre: forro year: 2022
Thread n° 7  -- Round n° 1  -- Request n° 37
genre:forro year: 2022
genre: dubstep year: 2022
Thread n° 6  -- Round n° 1  -- Request n° 31
genre:dubstep year: 2022
genre: goth year: 2022
Thread n° 8  -- Round n° 1  -- Request n° 43
genre:goth year: 2022
genre: hard-rock year: 2022
Thread n° 9  -- Round n° 1  -- Request n° 49
genre:hard-rock year: 2022
genre: industrial year: 2022
Thread n° 11  -- Round n° 1  -- Request n° 61
genre:industrial year: 2022
genre: acoustic year: 2022
Thread n° 1  -- Round n° 1  -- Request n° 1
genre:acoustic year: 2022
genre: honky-tonk year: 2022
Thread n° 10  -- Round n° 1  -- Request n° 55
genre:honky-tonk year: 2022
genre: b

In [62]:
# Displaying the length of:
#
# - track_db_without_af
# - af_db

print("Nombre de (tracks) récupérés :", len(track_db_without_af))
print("Nombre de listes de caractéristiques (audio features) récupérées :", len(af_db))

Nombre de (tracks) récupérés : 55970
Nombre de listes de caractéristiques (audio features) récupérées : 55970


In [63]:
# Saving data after execution of the above cell

pd.DataFrame(track_db_without_af).to_csv('data-out/musicbrainz/temp_backup/track_db_without_af___last.csv')
pd.DataFrame(af_db).to_csv('data-out/musicbrainz/temp_backup/af_db___last.csv')

In [64]:
# Loading the data (previously saved in the above cell)

track_df_without_af = pd.read_csv('data-out/musicbrainz/temp_backup/track_db_without_af___last.csv')
af_df = pd.read_csv('data-out/musicbrainz/temp_backup/af_db___last.csv')

In [65]:
# Merging dataframes track_df_without_af and af_df
# into a final dataframe >> track_df_with_af
#
# (dropping duplicates at the same time)

track_df_with_af = pd.DataFrame()

# OLD method (used if skipping the save and load)
#
# af_df = pd.DataFrame(af_db)
# track_df_without_af = pd.DataFrame(track_db_without_af)
# af_df.to_csv('af_df.csv')
# track_df_without_af.to_csv('track_df_without_af.csv')
# print(track_df_without_af.info(), '\n', track_df_without_af)

track_df_without_af.drop_duplicates(ignore_index=True, subset=['track_name', 'duration_ms'], keep = 'first', inplace = True)

print(len(af_df)," = ",len(track_df_without_af)," = ",len(track_df_with_af))
track_df_with_af = pd.merge(track_df_without_af, af_df, on='track_id', how='inner')
print(len(af_df)," = ",len(track_df_without_af)," = ",len(track_df_with_af))

55970  =  40851  =  0
55970  =  40851  =  51969


In [66]:
print(len(track_df_with_af))
track_df_with_af.drop_duplicates(ignore_index=True, subset=['track_name', 'duration_ms_x'], keep = 'first', inplace = True)
print(len(track_df_with_af))

51969
40851


In [67]:
track_df_with_af.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40851 entries, 0 to 40850
Data columns (total 28 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0_x      40851 non-null  int64  
 1   track_id          40851 non-null  object 
 2   track_name        40851 non-null  object 
 3   release_date      40851 non-null  object 
 4   popularity        40851 non-null  int64  
 5   duration_ms_x     40851 non-null  int64  
 6   explicit          40851 non-null  bool   
 7   is_local          40851 non-null  bool   
 8   track_number      40851 non-null  int64  
 9   genre             40851 non-null  object 
 10  Unnamed: 0_y      40851 non-null  int64  
 11  danceability      40851 non-null  float64
 12  energy            40851 non-null  float64
 13  key               40851 non-null  int64  
 14  loudness          40851 non-null  float64
 15  mode              40851 non-null  int64  
 16  speechiness       40851 non-null  float6

In [68]:
track_df_with_af.rename(columns={'duration_ms_x':'duration_ms'}, inplace=True)
track_df_with_af.drop('duration_ms_y', axis=1, inplace=True)
track_df_with_af.drop('Unnamed: 0_x', axis=1, inplace=True)
track_df_with_af.drop('Unnamed: 0_y', axis=1, inplace=True)

In [69]:
# Saving the final CSV file

seq_number = 1

while os.path.exists('data-out/musicbrainz/final/track_df_with_af_' + str(seq_number) + '.csv'):
    seq_number += 1

track_df_with_af.to_csv('data-out/musicbrainz/final/track_df_with_af_' + str(seq_number) + '.csv', index=False)

track_df_with_af.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40851 entries, 0 to 40850
Data columns (total 25 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   track_id          40851 non-null  object 
 1   track_name        40851 non-null  object 
 2   release_date      40851 non-null  object 
 3   popularity        40851 non-null  int64  
 4   duration_ms       40851 non-null  int64  
 5   explicit          40851 non-null  bool   
 6   is_local          40851 non-null  bool   
 7   track_number      40851 non-null  int64  
 8   genre             40851 non-null  object 
 9   danceability      40851 non-null  float64
 10  energy            40851 non-null  float64
 11  key               40851 non-null  int64  
 12  loudness          40851 non-null  float64
 13  mode              40851 non-null  int64  
 14  speechiness       40851 non-null  float64
 15  acousticness      40851 non-null  float64
 16  instrumentalness  40851 non-null  float6

In [52]:
track_df_with_af_2022 = track_df_with_af[track_df_with_af['release_date'].str.contains('2022')]

In [55]:
track_df_with_af_2022.to_csv('2023.03.01_extract_2022.csv')

In [56]:
track_df_with_af_2022['genre'].value_counts()

black-metal    1379
bluegrass      1344
breakbeat      1328
brazil         1292
psych-rock     1240
               ... 
latin           106
indie-pop        74
house            70
latino           67
songwriter        8
Name: genre, Length: 113, dtype: int64

In [70]:
# Concaténation de tous les CSVs récupérés

pd.concat([pd.read_csv('data-out/musicbrainz/final/track_df_with_af_1.csv'),
           pd.read_csv('data-out/musicbrainz/final/track_df_with_af_2.csv'),
           pd.read_csv('data-out/musicbrainz/final/track_df_with_af_3.csv'),
           pd.read_csv('data-out/musicbrainz/final/track_df_with_af_4.csv'),
           pd.read_csv('data-out/musicbrainz/final/track_df_with_af_5.csv'),
           pd.read_csv('data-out/musicbrainz/final/track_df_with_af_6.csv'),
           pd.read_csv('data-out/musicbrainz/final/track_df_with_af_7.csv'),
           pd.read_csv('data-out/musicbrainz/final/track_df_with_af_8.csv'),
           pd.read_csv('data-out/musicbrainz/final/track_df_with_af_9.csv')]).to_csv('data-out/musicbrainz/final/final_track_df_with_af.csv', index=False)

In [71]:
final_track_df_with_af = pd.read_csv('data-out/musicbrainz/final/final_track_df_with_af.csv')

In [72]:
# Un peu d'analyse des résultats

final_track_df_with_af.info()

# datetime.datetime.strptime(final_track_df_with_af['release_date'], '%Y-%m-%d').date().year == 2022


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 722436 entries, 0 to 722435
Data columns (total 25 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   track_id          722436 non-null  object 
 1   track_name        722436 non-null  object 
 2   release_date      722436 non-null  object 
 3   popularity        722436 non-null  int64  
 4   duration_ms       722436 non-null  int64  
 5   explicit          722436 non-null  bool   
 6   is_local          722436 non-null  bool   
 7   track_number      722436 non-null  int64  
 8   genre             722436 non-null  object 
 9   danceability      722436 non-null  float64
 10  energy            722436 non-null  float64
 11  key               722436 non-null  int64  
 12  loudness          722436 non-null  float64
 13  mode              722436 non-null  int64  
 14  speechiness       722436 non-null  float64
 15  acousticness      722436 non-null  float64
 16  instrumentalness  72

In [76]:
print(len(final_track_df_with_af))
final_track_df_with_af.drop_duplicates(ignore_index=True, subset=['track_name', 'duration_ms'], keep = 'first', inplace = True)
print(len(final_track_df_with_af))

51800
51800


C:\Users\cduro\AppData\Local\Temp\ipykernel_13980\1926273003.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_track_df_with_af.drop_duplicates(ignore_index=True, subset=['track_name', 'duration_ms'], keep = 'first', inplace = True)


In [75]:
final_track_df_with_af = final_track_df_with_af[final_track_df_with_af['release_date'].str.contains('2022')]

In [ ]:
# Test Cell

list(range(5))
list(range(5,10))
list(range(10,15))

In [ ]:
# Test Cell

test = spotify_client[0].search(q='year:2013 genre:acoustic', type='track', limit=50)
len(pd.DataFrame(test)['tracks']['items']) #[16]['id']

In [ ]:
# '41ERrwfzos93Xlf6hFBiDn'
# album_tracks(album_id, limit=50, offset=0, market=None)

test = spotify_client[0].album_tracks('1l4VD485oDSJM23eWJEWJN', limit=50, offset=0)
test

In [3]:
df_artist_name_and_album_id = pd.read_csv('../data/data_110k_lignes.csv', usecols=['artist_name', 'album_id'])

In [4]:
print(len(df_artist_name_and_album_id))
df_artist_name_and_album_id.drop_duplicates(inplace=True)
print(len(df_artist_name_and_album_id))

110158
41895


In [5]:
df_artist_name_and_album_id[:10]

,artist_name,album_id
0,Gen Hoshino,41ERrwfzos93Xlf6hFBiDn
1,Ray LaMontagne,7cwHkTgTHLbCUFB6hRi9QQ
2,Steve Moakler,457Ok9pnF5fAfG7lNUFqHS
3,Ben Rector,1l4VD485oDSJM23eWJEWJN
5,Drew Holcomb,6fuKDHZ9UfLmssjdRTjPfz
6,Penny and Sparrow,4ruy6zMB9t2sKrNyK2B8zF
8,Andrew Ripp,3EcsslHYdm8UlzuwnvaA9O
9,Ben Rector,36jkSaatEARkByil5kn9f6
10,Zack Tabudlo,0HNCZzy2tqjuLZJuTDGdAW
11,Zack Tabudlo,0MK93ibEen72O8dosu3WXE


In [6]:
input_data = df_artist_name_and_album_id

client_id = ['02dc5f25b96f4315a7182583264c75ba','7672d38906344d66bf7bc384e290950a','82a2058cd6d0435bb464648fdcf5dc8a','e567bb78a9ea46deb253263bfb4b0cc8','810337a602d14bd884429991ad417eba','3e931759ccea4d49bcaf44a1e4f6059b','6b353a3c6abe48fe86564875faecbb09','73ebc19135a54e4c9d9d7bae6a798aea','085733e14f894738bf40334443020126', 'bf7349672eab4050bf4567ed7c6732ec', 'dbf80683949a456592837fe493b579a7', 'b6f601f082f7491aa9c4ab7271fae6fd', '0464cab38a1a4fb89c8678f6b1e27e81', '65df35b84b8a47d2b66e21a2a8e39dd3', 'f61219e4380f4b8796e9f4d224da5494', '3c51ca3a9e314bd5aeb608d22e369efd', '3f3a6839a5de407ab16c87ba3ba8cd68', 'a8461dc61ff44abd83f94a4a07de530f', '7a06f1d458ac41f8b25577e02ec30d62']
client_secret = ['1708c607af8844e2aa4c8948d5098422','f47c320326924302a7e17995cd7b6aad','88549dde8bd04284a8c380d34fba4468','bebddff9b80841529c008b87fb9bf1e1','1eef810711464eafa0079ce3f48ae921','a1925f6f589f43e7b4456665ef0feb21','3f598249bd864a24bd07344adf9d9ac8','2ddf3a4559884ce6a2b438a0ec3c2937','62d8f8eccfdf49d9bf8710eb47d2a34c', 'bda8d2acbf974cd5932f00a6a763fad3', 'ac2a80f3cde94feb8c4b886cfd556cfd', 'fc0e21ebc5974886acc6ba14d4676c02', '58c42925f46941c6b7ca21cb157908b6', '90e3ed69a2f24735a5d4710bc6a377fa', 'cb4cb64077834ba2bae059bf983dd47f', '174b4c06e0974b01bbf7294d11c9bd7f', '03410f4b5165403b8219d14cc9550605', 'ed2e1d6898834e4bb649df080e5474bf', '2f10949260a449968cb0fa13f1e5a4c9']

spotify_client_credentials = []
spotify_client = []

for i in range(len(client_id)):
      spotify_client_credentials.append(SpotifyClientCredentials(client_id=client_id[i], client_secret=client_secret[i]))

for cred in spotify_client_credentials:                                                    
      spotify_client.append(spotipy.Spotify(client_credentials_manager = spotify_client_credentials[0]))

number_of_threads = len(spotify_client)

def get_tracks_from_albums(start, end, thread_number):
    df_result = pd.DataFrame()
    
    artist_names = list(input_data['artist_name'])
    album_ids = list(input_data['album_id'])

    number_of_slices = len(input_data) // 20 + 1
    
    slice_size = len(input_data) // (number_of_threads)
    slice_limit = []

    for i in range(number_of_threads + 1):
        if i == 0:
            slice_limit.append(0)
        elif i != number_of_threads:
            slice_limit.append(i*slice_size)
        else:
            slice_limit.append(len(input_data))

    test
    
    return df_result

print(get_tracks_from_albums(input_data))

TypeError: get_tracks_from_albums() missing 2 required positional arguments: 'end' and 'thread_number'